In [1]:
import os, subprocess
import time
import numpy as np
from pprint import pprint
from collections import OrderedDict

from simtk import unit
from simtk.openmm import app, LangevinIntegrator
from simtk.openmm.app import PDBFile

import parmed as pmd
from parmed.utils.six import string_types

import matplotlib.pyplot as plt
import mpld3

from pmx import *
from pmx.forcefield import *

# Choose underlying toolkits for the OpenFF toolkit
Choose the toolkit(s) you want to use with the OpenFF toolkit (either OpenEye or Ambertools and RDKit)

In [2]:
from openforcefield.utils import toolkits

### OpenEye version: uncomment the following if you have and if you want to use the OpenEye toolkit, then RDKit and Ambertools toolkits
# toolkit_precedence = [toolkits.OpenEyeToolkitWrapper, toolkits.RDKitToolkitWrapper, toolkits.AmberToolsToolkitWrapper]

### Non-OpenEye version: uncomment the following if you want to use the rdkit and ambertools
toolkit_precedence = [toolkits.RDKitToolkitWrapper, toolkits.AmberToolsToolkitWrapper]

toolkits.GLOBAL_TOOLKIT_REGISTRY = toolkits.ToolkitRegistry(toolkit_precedence=toolkit_precedence)

In [3]:
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField
from openforcefield.tests.utils import compare_system_parameters, compare_system_energies

In [4]:
# choose the force field:
# not yet the release version!
forcefield = 'smirnoff99Frosst-1.1.0.offxml'
openff_forcefield = ForceField(forcefield)

In [5]:
# function to convert openFF molecule (ligand) to a parmed structure
def ligandToPMD(ligand):
    ligand_positions = ligand.conformers[0]
    
    # Calculate am1bcc charges and fix them such that they add up to zero (4 digit precision)
    try:
        ligand.compute_partial_charges_am1bcc()
    except Exception as e:
        raise Exception('Error in charge calculation for ligand {}: {}'.format(ligand.name, e))
    # Give all atoms unique names so we can export to GROMACS
    for idx, atom in enumerate(ligand.atoms):
        atom.name = f'{atom.element.symbol}{idx}'
    
    # Do not assign H-bond constraints now, instead have ParmEd add them later
    del openff_forcefield._parameter_handlers['Constraints']

    ligand_topology = ligand.to_topology()
    try:
        ligand_system = openff_forcefield.create_openmm_system(ligand_topology, charge_from_molecules=[ligand])
    except Exception as e:
        raise Exception('Error in creating openmm system: {}'.format(e))
    # Create OpenMM Topology from OpenFF Topology
    omm_top = ligand_topology.to_openmm()


    # Convert OpenMM System to a ParmEd structure.
    pmd_structure = pmd.openmm.load_topology(omm_top, ligand_system, ligand_positions)

    return pmd_structure, ligand_topology, ligand_system, ligand_positions

In [6]:
# function for analyzing energy contributions of openMM energies
def forcegroupify(system):
    forcegroups = {}
    for i in range(system.getNumForces()):
        force = system.getForce(i)
        force.setForceGroup(i)
        forcegroups[(type(force)).__name__] = i
    return forcegroups

def getEnergyDecomposition(context, forcegroups):
    energies = {}
    for f, i in forcegroups.items():
        energies[f] = context.getState(getEnergy=True, groups=2**i).getPotentialEnergy()
    energies['TotalPotential'] = context.getState(getEnergy=True).getPotentialEnergy()
    return energies

In [7]:
# functions for Gromacs force field  manuplation/conversion
def set_charge_to_zero(itp):
    q = 0.0
    n = 0
    for a in itp.atoms:
        q += a.q
        n += 1
    intq = round(q)
    diffq = intq - q
    # round to 6 digit precision
    deltaq = np.around(diffq/float(n), decimals=6)
    
    newq = 0.0
    for a in itp.atoms:
        a.q += deltaq
        a.q = np.around(a.q, decimals=6)
        newq += a.q
    # add remainder to first atom
    intq = round(newq)
    diffq = intq - newq
    itp.atoms[0].q += diffq

def change_atomtypes(itp,suffix):
    for a in itp.atoms:
#        newtype = str(a.atomtype)+'_'+str(a.name)+str(suffix)
        newtype = str(a.atomtype)+str(suffix)
        a.atomtype = newtype

    newdict = {}
    print(itp.atomtypes)
    for atkey in itp.atomtypes.keys():
        print(atkey)
        at = itp.atomtypes[atkey]
#        newtype = str(atkey)+'_'+str(at[0])+str(suffix)
        newtype = str(atkey)+str(suffix)
        newdict[newtype] = at
    itp.atomtypes=newdict

def write_ff(atypes, fname, ff='amber99sb'):
    fp = open(fname,'w')
    print('[ atomtypes ]', file=fp)
    for atkey in atypes.keys():
        at = atypes[atkey]
        print('%8s %12.6f %12.6f %3s %12.6f %12.6f' % (atkey, at[1], at[2], at[3], at[4], at[5]), file=fp)

def write_posre(itp, fname, fc=1000):
    fp = open(fname,'w')
    print('[ position_restraints ]', file=fp)
    for i, atom in enumerate(itp.atoms):
        print("%d   1    %d   %d    %d" % (i+1,fc,fc,fc), file=fp)
    fp.close()


# Convert SDF files to gromacs topologies

In [9]:
omm_systems = {}

If you don't have SDF files, you first need to convert them (i.e. with the OpenEye toolkit OEChem)

ATTENTION: Using PDB files might prone to errors because the pdb files do not have bond information

In [11]:
import shutil
for target_id, target in enumerate(['jnk1', 'pde2', 'thrombin']):
    print('=== ' + target + ' ===')
    print(f'./input/{target_id+1:02d}_{target}/ligands.txt')
    with open(f'./input/{target_id+1:02d}_{target}/ligands.txt') as f:
        ligNames = f.read().splitlines()
        for ligName in ligNames:
            ligPathOld= 'systems/' + target + '/' + ligName + '/' 
            ligPathNew= f'systems/{target_id+1:02d}_{target}/03_docked/' + ligName + '/' 
            os.makedirs(ligPathNew, exist_ok = True)
            shutil.copy(f'{ligPathOld}/{ligName}.pdb', f'{ligPathNew}/')
            shutil.copy(f'{ligPathOld}/{ligName}.sdf', f'{ligPathNew}/')

=== jnk1 ===
./input/01_jnk1/ligands.txt
=== pde2 ===
./input/02_pde2/ligands.txt
=== thrombin ===
./input/03_thrombin/ligands.txt


In [10]:
for target_id, target in enumerate(['jnk1', 'pde2', 'thrombin']):
    print('=== ' + target + ' ===')
    omm_systems[target] = {}
    with open(f'./input/{target_id+1:02d}_{target}/ligands.txt') as f:
        ligNames = f.read().splitlines()
    for ligName in ligNames:
        ligPath= f'systems/{target_id+1:02d}_{target}/03_docked/{ligName}/'
        topPath= f'systems/{target_id+1:02d}_{target}/04_topo/{forcefield}/{ligName}/'  
        os.makedirs(f'{topPath}', exist_ok=True)
        
        if os.path.isfile(f'{ligPath}/{ligName}.sdf'):
            ligand = Molecule.from_file(f'{ligPath}/{ligName}.sdf')
        elif os.path.isfile(f'{ligPath}/{ligName}.pdb'):
            # Try to read in PDB file instead of a SDF, only works with OpenEye
            ligand = Molecule.from_file(f'{ligPath}/{ligName}.pdb')
            # save as sdf file 
            # ATTENTION: automatic conversion to SDF
            ligand.to_file(f'{ligPath}/{ligName}.sdf', 'sdf')
        else:
            print('    File not found. Molecules {} cannot be read in'.format(ligName))
            continue
        
        print('   ', ligName)  
        try: 
            pmd_structure, ligand_topology, ligand_system, ligand_positions = ligandToPMD(ligand)
        except Exception as e:
            print('    ' + str(e))
            continue
        
        omm_systems[target][ligName] = ligand_system
    
        # Export GROMACS files.
        pmd_structure.save(f'{topPath}/{ligName}.top', overwrite = True)
        pmd_structure.save(f'{topPath}/{ligName}.gro', overwrite = True, precision = 8)

        # Export AMBER files.
        pmd_structure.save(f'{topPath}/{ligName}.prmtop', overwrite=True)
        pmd_structure.save(f'{topPath}/{ligName}.inpcrd', overwrite=True){target}/03_docked/{ligName}/'
        topPath= f'systems/{target}/04_topo/{forcefield}/{ligName}/'  
        os.makedirs(f'{topPath}', exist_ok=True)
        
        if os.path.isfile(f'{ligPath}/{ligName}.sdf'):
            ligand = Molecule.from_file(f'{ligPath}/{ligName}.sdf')
        elif os.path.isfile(f'{ligPath}/{ligName}.pdb'):
            # Try to read in PDB file instead of a SDF, only works with OpenEye
            ligand = Molecule.from_file(f'{ligPath}/{ligName}.pdb')
            # save as sdf file 
            # ATTENTION: automatic conversion to SDF
            ligand.to_file(f'{ligPath}/{ligName}.sdf', 'sdf')
        else:
            print('    File not found. Molecules {} cannot be read in'.format(ligName))
            continue
        
        print('   ', ligName)  
        try: 
            pmd_structure, ligand_topology, ligand_system, ligand_positions = ligandToPMD(ligand)
        except Exception as e:
            print('    ' + str(e))
            continue
        
        omm_systems[target][ligName] = ligand_system
    
        # Export GROMACS files.
        pmd_structure.save(f'{topPath}/{ligName}.top', overwrite = True)
        pmd_structure.save(f'{topPath}/{ligName}.gro', overwrite = True, precision = 8)

        # Export AMBER files.
        pmd_structure.save(f'{topPath}/{ligName}.prmtop', overwrite=True)
        pmd_structure.save(f'{topPath}/{ligName}.inpcrd', overwrite=True)
        
        # Create GROMACS ITP file
        subprocessOutput = subprocess.run(f'python ./scriptpath/top2itp.py \
                                                {topPath}/{ligName}.top '.split(), 
                                               capture_output = True, text = True)
        print(subprocessOutput.stdout.split('\n') + subprocessOutput.stderr.split('\n'))
        subprocess.run(f'mv MOL.itp ffMOL.itp posre_MOL.itp {topPath}/'.split(), 
                                               capture_output = True, text = True)
            
        itp = read_gaff_top(f'{topPath}/{ligName}.top')
        itp.set_name('MOL')
        print(itp.atomtypes)
        change_atomtypes(itp, ligName)
        set_charge_to_zero(itp)

        itpout = ligName + '.itp'
        posre = 'posre_' + ligName + '.itp'
        itp.write(f'{topPath}/{itpout}')
        print(itp.atomtypes)
        write_ff(itp.atomtypes, f'{topPath}/ff{itpout}')
        write_posre(itp, f'{topPath}/ff{posre}') 

=== jnk1 ===
    lig_17124-1
    lig_18624-1
    lig_18625-1
    lig_18626-1
    lig_18627-1
    lig_18628-1
    lig_18629-1
    lig_18630-1
    lig_18631-1
    lig_18632-1
    lig_18633-1
    lig_18634-1
    lig_18635-1
    lig_18636-1
    lig_18637-1
    lig_18638-1
    lig_18639-1
    lig_18652-1
    lig_18658-1
    lig_18659-1
    lig_18660-1
=== pde2 ===
    lig_43249674
    lig_48009208
    lig_48022468
    lig_48168913
    lig_48271249
    lig_49072088
    lig_49137374
    lig_49137530
    lig_49175789
    lig_49175828
    lig_49220392
    lig_49220548
    lig_49396360
    lig_49580115
    lig_49582390
    lig_49582468
    lig_49585367
    lig_49932129
    lig_49932714
    lig_50107616
    lig_50181001
=== thrombin ===
    lig_1a
    lig_1b
    lig_1c
    lig_1d
    lig_3a
    lig_3b
    lig_5
    lig_6a
    lig_6b
    lig_6e
    lig_7a


In [14]:
for target_id, target in enumerate(['jnk1', 'pde2', 'thrombin']):
    print('=== ' + target + ' ===')
    omm_systems[target] = {}
    with open(f'./input/{target_id+1:02d}_{target}/ligands.txt') as f:
        ligNames = f.read().splitlines()
    for ligName in ligNames:
        topPath= f'systems/{target_id+1:02d}_{target}/04_topo/{forcefield}/{ligName}/'  
        if os.path.isfile(f'{topPath}/{ligName}.top'):
            print('   ', ligName)
            subprocessOutput = subprocess.run(f'python ./scriptpath/top2itp.py \
                                                {topPath}/{ligName}.top '.split(), 
                                               capture_output = True, text = True)
            print(subprocessOutput.stdout.split('\n') + subprocessOutput.stderr.split('\n'))
            for file in ['MOL.itp', 'ffMOL.itp', 'posre_MOL.itp']:
                shutil.copy(file, topPath)
                
            itp = read_gaff_top(f'{topPath}/{ligName}.top')
            itp.set_name('MOL')
            print(itp.atomtypes)
            change_atomtypes(itp, ligName)
            set_charge_to_zero(itp)

            itpout = ligName + '.itp'
            posre = 'posre_' + ligName + '.itp'
            itp.write(f'{topPath}/{itpout}')
            print(itp.atomtypes)
            write_ff(itp.atomtypes, f'{topPath}/ff{itpout}')
            write_posre(itp, f'{topPath}/ff{posre}') 

=== jnk1 ===
    lig_17124-1
['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'Br1': ['35', 79.9041, 0.0, 'A', 0.39555903, 1.33888], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'Br1': ['35', 79.9041, 0.0, 'A

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'S1': ['16', 32.0655, 0.0, 'A', 0.35635949, 1.046], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'S1': ['16', 32.0655, 0.0, 'A', 0.35635949, 1.046], 'H1': ['1'

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'S1': ['16', 32.0655, 0.0, 'A', 0.35635949, 1.046], 'O1': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H3': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888], 'H4': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'S1': ['16', 32.0655, 0.0, 'A', 0.35635949, 1.046], 'O1': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'O2': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C3': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.87864], 'H1': ['1'

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.25105526, 0.06276], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.25105526, 0.06276], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888]}
C1
O1
C2
N1
H1
H2
H3
H4
{'C1lig_48271249': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1lig_48271249': ['

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'Cl1': ['17', 35.4532, 0.0, 'A', 0.34709414, 1.10876]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'H1': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'Cl1': ['17', 35.4532, 0.0, 'A', 0.34709414, 1.10876]}
C1
O1
C2
N1
H1
H2
H3
Cl1
{'C1lig_49220392': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1lig_49220392':

['', '']
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.25105526, 0.06276], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888]}
{'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1': ['8', 15.99943, 0.0, 'A', 0.30000123, 0.71128], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.25105526, 0.06276], 'H3': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'H4': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888]}
C1
O1
C2
N1
H1
H2
H3
H4
{'C1lig_49585367': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'O1lig_49585367': ['

['', '']
{'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'O1': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.19599772, 0.0656888], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888], 'H5': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276], 'Cl1': ['17', 35.4532, 0.0, 'A', 0.34709414, 1.10876]}
{'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'O1': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.19599772, 0.0656888], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.065688

['', '']
{'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'O1': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.19599772, 0.0656888], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888], 'H5': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276]}
{'N1': ['7', 14.00672, 0.0, 'A', 0.32499985, 0.71128], 'C1': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.4577296], 'C2': ['6', 12.01078, 0.0, 'A', 0.33996695, 0.359824], 'O1': ['8', 15.99943, 0.0, 'A', 0.29599219, 0.87864], 'H1': ['1', 1.007947, 0.0, 'A', 0.2471353, 0.0656888], 'H2': ['1', 1.007947, 0.0, 'A', 0.26495328, 0.0656888], 'H3': ['1', 1.007947, 0.0, 'A', 0.19599772, 0.0656888], 'H4': ['1', 1.007947, 0.0, 'A', 0.10690785, 0.0656888], 'H5': ['1', 1.007947, 0.0, 'A', 0.25996425, 0.06276

# Generate GAFF2 topologies

In [20]:
forcefield = 'gaff2'
for target_id, target in enumerate(['jnk1', 'pde2', 'thrombin']):
    print('=== ' + target + ' ===')
    with open(f'./input/{target_id+1:02d}_{target}/ligands.txt') as f:
        ligNames = f.read().splitlines()
    for ligName in ligNames:
        ligPath= f'systems/{target_id+1:02d}_{target}/03_docked/{ligName}/'
        topPath= f'systems/{target_id+1:02d}_{target}/04_topo/{forcefield}/{ligName}/'  
        os.makedirs(f'{topPath}', exist_ok=True)
        
        if os.path.isfile(f'{ligPath}/{ligName}.pdb'):
            os.makedirs(f'{topPath}', exist_ok=True)
            # quick hack to get formal charge of molecule
            itp = read_gaff_top(f'systems/{target_id+1:02d}_{target}/04_topo/smirnoff99Frosst-1.1.0.offxml/{ligName}/{ligName}.top')
            intq = round(sum([a.q for a in itp.atoms]))
            print('Formal Charge:', intq)
            subprocessOutput = subprocess.run(f'python ./scriptpath/acpype.py \
                                                -i {ligPath}/{ligName}.pdb \
                                                -o gmx \
                                                -a gaff2 \
                                                -n {int(intq)}'.split(),
                                                capture_output = True, text = True)
            print(subprocessOutput.stdout.split('\n') + subprocessOutput.stderr.split('\n'))
            shutil.copy(f'{ligName}.acpype/{ligName}_GMX.itp', topPath)
            shutil.copy(f'{ligName}.acpype/{ligName}_GMX.gro', topPath)
            shutil.copy(f'{ligName}.acpype/{ligName}_GMX.top', topPath) 
            
            if os.isdir(f'{topPath}/{ligName}.acpype'):
                shutil.rmtree(f'{topPath}/{ligName}.acpype')
            shutil.move(f'{ligName}.acpype', f'{topPath}')
            
            subprocessOutput = subprocess.run(f'python ./scriptpath/top2itp.py \
                                                {topPath}/{ligName}_GMX.itp '.split(), 
                                               capture_output = True, text = True)
            print(subprocessOutput.stdout.split('\n') + subprocessOutput.stderr.split('\n'))
            for file in ['MOL.itp', 'ffMOL.itp', 'posre_MOL.itp']:
                shutil.copy(file, topPath)

=== jnk1 ===
Formal Charge: -0.0
['============================================================================', '| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |', '============================================================================', '==> ... charge set to 0', '==> ... converting pdb input file to mol2 input file', '==> * Babel OK *', '==> Executing Antechamber...', '==> * Antechamber OK *', '==> * Parmchk OK *', '==> Executing Tleap...', '++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++', "Checking 'LIG'....", "Checking parameters for unit 'LIG'.", 'Checking for bond parameters.', 'Checking for angle parameters.', 'Unit is OK.', '++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++', '==> * Tleap OK *', '==> Removing temporary files...', '==> Writing GROMACS files', '', '==> Writing GMX dihedrals for GMX 4.5 and higher.', '', '==> Writing pickle file lig_17124-1.pkl', 'Total time of exe

FileNotFoundError: [Errno 2] No such file or directory: 'systems/01_jnk1/04_topo/gaff2/lig_18625-1//lig_18625-1.acpype'

# Compare the different topologies by comparing parameters calculating and comparing OpenMM energies

In [ ]:
for target in ['testmols',  'jnk1', 'pde2', 'thrombin', 'pde2_sdf']:
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    ligands = ! ls -d 'systems/'$target/*/
    for lig in ligands:
        lig = lig.split('/')
        ligand = lig[-2]
        ligPath = 'systems/' + target + '/' + ligand + '/' + ligand
        print('-' * 60)
        print('--- ' + ligand + ' ---')
        print('-' * 60)
        
        off_mol = Molecule.from_file(ligPath + '.sdf', 'sdf')
        positions = off_mol.conformers[0]
    
        # load GROMACS topology and create an OpenMM system object
        if os.path.isfile(ligPath+'.top'):           
            gromacs_topology = pmd.load_file(ligPath + '.top')
            gromacs_system = gromacs_topology.createSystem(nonbondedMethod = app.NoCutoff, 
                                                           removeCMMotion = False)

            
        # load AMBER topology; the following is adapted from 
        # https://github.com/openforcefield/openforcefield/blob/master/examples/using_smirnoff_in_amber_or_gromacs/convert_to_amber_gromacs.ipynb
        if os.path.isfile(ligPath+'.prmtop'):            
            # Load the prmtop files into a ParmEd Structure and create an OpenMM System object.
            amber_topology = pmd.load_file(ligPath + '.prmtop')

            amber_system = amber_topology.createSystem(nonbondedMethod = app.NoCutoff,
                                                       removeCMMotion=False)
            
        # Compare the parameters of the original and converted Systems.
        # This raises FailedParameterComparisonError if the comparison fails.
        try:
            compare_system_parameters(omm_systems[target][ligand], gromacs_system)
            compare_system_parameters(omm_systems[target][ligand], amber_system)
        except Exception as e:
            print(f'Failed to compare systems: {e}')

        # Compare the energies by force.
        # This raises FailedEnergyComparisonError if the comparison fails.
        try:
            omm_energies, gromacs_energies = compare_system_energies(
                omm_systems[target][ligand], gromacs_system, positions, rtol=1e-3)
            omm_energies, amber_energies = compare_system_energies(
                omm_systems[target][ligand], amber_system, positions, rtol=1e-3)
        except Exception as e:
            print(f'Failed to compare energies: {e}')
            
        pprint(omm_energies)
        pprint(amber_energies)
        pprint(gromacs_energies)

# Run single point energies and minimizations in openMM and Gromacs

The following cell contains a function and dictionaries to canonicalize the energy output of different MD engines. Adapted from https://github.com/shirtsgroup/InterMol
    

In [ ]:
canonical_energy_names = [
    'potential',
        'bonded',
            'bond',

            'angle', 'urey-bradley',

            'dihedral', 
                'improper', 
                'proper', 
                'cmap',

        'nonbonded',
            'h-bond',
            'vdw total', 
                'vdw-14', 
                'vdw (SR)', 
                'vdw (LR)',
            'coulomb total', 
                'coulomb-14', 
                'coulomb (SR)', 
                'coulomb (LR)',
    ]

canonical_indent = [
    0,
        1,
            2,
            2,
                3,
            2,
                3,
                3,
                3,
        1,
            2,
            2,
                3,
                3,
                3,
            2,
                3,
                3,
                3
]

def canonicalize_energy_names(energy_dict, canonical_keys):
    """Adjust the keys in energy_dict to the canonical names.

    Parameters
    ----------
    energy_dict : dict
    engine : str

    Returns
    -------
    normalized : dict

    """
    normalized = OrderedDict.fromkeys(canonical_energy_names,
                                      0 * unit.kilojoules_per_mole)

    for key, energy in energy_dict.items():
        canonical_key = canonical_keys.get(key)
        if canonical_key is None:
            continue
        elif not isinstance(canonical_key, string_types):
            for k in canonical_key:
                normalized[k] += energy.in_units_of(unit.kilojoules_per_mole)
        else:
            normalized[canonical_key] += energy.in_units_of(unit.kilojoules_per_mole)

    if 'Non-bonded' in canonical_keys:
        normalized['nonbonded'] = energy_dict['Non-bonded']
    elif  'NonbondedForce' in canonical_keys:
        normalized['nonbonded'] = energy_dict['NonbondedForce']
    else:
        normalized['nonbonded'] = normalized['vdw total'] + normalized['coulomb total'] + normalized['h-bond']

    # could be a problem here since desmond calls UB angle as stretch = bond
    normalized['bonded'] = (normalized['bond'] + normalized['angle'] + normalized['dihedral'])

    return normalized

amber_to_canonical = {
    'BOND': ['bond'],

    'ANGLE': ['angle'],
    'UB': ['angle','urey-bradley'],

    'DIHED': ['dihedral','proper'],
    'IMP': ['dihedral','improper'],
    'CMAP': ['dihedral','cmap'],

    'HBOND': ['h-bond'],

    'VDWAALS': ['vdw total'],
    '1-4 VDW': ['vdw total', 'vdw-14'],

    'EEL': ['coulomb total'],
    '1-4 EEL': ['coulomb total', 'coulomb-14'],

    'ENERGY': ['potential']
}

gromacs_to_canonical = {
    'Bond': 'bond',

    'Angle': 'angle',
    'G96Angle': 'angle',
    'Restricted Angles': 'angle',
    'Bond-Cross': 'angle',
    'BA-Cross': 'angle',
    'Quartic Angles': 'angle',
    'U-B': ['urey-bradley','angle'],

    'Proper Dih.': ['dihedral', 'proper'],
    'Ryckaert-Bell.': ['dihedral', 'proper'],
    'Improper Dih.': ['dihedral', 'improper'],
    'CMAP': ['dihedral','cmap'],
    'LJ (SR)': ['vdw total', 'vdw (SR)'],
    'LJ-14': ['vdw total','vdw-14'],
    'LJ recip.': ['vdw total', 'vdw (LR)'],
    'Disper. corr.': ['vdw total', 'vdw (LR)'],
    'Coulomb (SR)': ['coulomb total','coulomb (SR)'],
    'Coulomb-14': ['coulomb total','coulomb-14'],
    'Coul. recip.': ['coulomb total','coulomb (LR)'],
    'Potential': ['potential']
}

openmm_to_canonical = {
    'HarmonicBondForce': 'bond',

    'HarmonicAngleForce': 'angle',

    'PeriodicTorsionForce': ['dihedral','proper'],
    
    'ImproperTorsionForce': ['dihedral','improper'],

    'NonbondedForce': 'nonbonded',
    
    'TotalPotential': 'potential'
}

#### openMM

Here, we run the single point energy calculations and minimizations in openMM. It is important that we use *exactly* the same conformations (i.e. the same precision) to avoid discrepancies in the energies later due to rounding. That's why we take the GRO file coordinates.

In [ ]:
openmmEnergies = {}
openmmMinimizedEnergies = {}
for target in ['testmols',  'jnk1', 'pde2', 'thrombin', 'pde2_sdf']:
    openmmEnergies[target] = {}
    openmmMinimizedEnergies[target] = {}
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    ligands = ! ls -d 'systems/'$target/*/
    for lig in ligands:
        lig = lig.split('/')
        ligand = lig[-2]
        ligPath = 'systems/' + target + '/' + ligand + '/' + ligand
        
        if os.path.isfile(ligPath+'.gro'):
            print('-' * 60)
            print('--- ' + ligand + ' ---')
            print('-' * 60)
            gro_positions = pmd.load_file(ligPath + '.gro')
            
            ligand_system = omm_systems[target][ligand]

            integrator = LangevinIntegrator(300*unit.kelvin, 
                            1/unit.picosecond, 
                            0.002*unit.picoseconds)

             
            simulation = app.Simulation(ligand_topology, ligand_system, integrator)
        
            simulation.context.setPositions(gro_positions.positions)
        
            energies = getEnergyDecomposition(simulation.context, forcegroupify(ligand_system))
            
            canonical = canonicalize_energy_names(energies, openmm_to_canonical)
            openmmEnergies[target][ligand] = canonical
            pprint(canonical, indent = 4)
    
            simulation.minimizeEnergy()
            minimized_coords = simulation.context.getState(getPositions=True).getPositions()
            
            energies = getEnergyDecomposition(simulation.context, forcegroupify(ligand_system))
            canonical = canonicalize_energy_names(energies, openmm_to_canonical)
            openmmMinimizedEnergies[target][ligand] = canonical

#### Gromacs

Run Gromacs to get energies. It is important to have the option -maxwarn 1 in grompp, because Gromacs sometimes gives warnings about unconstrained quickly oscillating bonds and consequently does not write files. It is also important to turn off constraints in the Gromacs input files - otherwise the energies are not comparable.

In [ ]:
def printMessages(subprocessOutput):
    output = subprocessOutput.stdout.split('\n') + subprocessOutput.stderr.split('\n')
    for i, line in enumerate(output):
        if line.startswith('NOTE') or line.startswith('WARNING') or line.startswith('ERROR'):
            print('    | ' + line)
            for ll in output[i+1:]:
                print('    | ' + ll)
                if not ll:
                    break

In [ ]:
zero = 0.0 * unit.kilojoules_per_mole
gromacsEnergies = {}
gromacsMinimizedEnergies = {}
for target in ['testmols',  'jnk1', 'pde2', 'thrombin', 'pde2_sdf']:
    gromacsEnergies[target] = {}
    gromacsMinimizedEnergies[target] = {}
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    ligands = ! ls -d 'systems/'$target/*/
    for lig in ligands:
        lig = lig.split('/')
        ligand = lig[-2]
        ligPath = 'systems/' + target + '/' + ligand + '/'
        outPath = 'systems/' + target + '/' + ligand + '/gromacs/'
        
        if os.path.isfile(ligPath+ligand+'.gro') and os.path.isfile(ligPath+ligand+'.top'):
            print('-' * 60)
            print('--- ' + ligand + ' ---')
            print('-' * 60)
            ! mkdir -p $outPath
            ! rm -f $outPath/*.*
            ! rm -f $ligPath/\#* $outPath/\#*
            
            
            # single point energy of starting structure
            energies = ''
            count = 0
            while not energies and count < 10:
                print('    Calculate single-point energy ...')
                print('    Message of gmx gromp:')
                grompp_output = subprocess.run(f'gmx grompp -f ./input/gromacs/md.mdp \
                                               -c {ligPath}/{ligand}.gro \
                                               -p {ligPath}/{ligand}.top \
                                               -o {outPath}/{ligand}_md.tpr \
                                               -maxwarn 1'.split(), 
                                               capture_output = True, text = True)
                printMessages(grompp_output)
                                
                print('    Messages of gmx mdrun:')
                mdrun_output = subprocess.run(f'gmx mdrun  -s {outPath}/{ligand}_md.tpr \
                                              -rerun {ligPath}/{ligand}.gro \
                                              -e {outPath}/{ligand}_md.edr \
                                              -g {outPath}/{ligand}_md.log'.split(), 
                                              capture_output = True, 
                                              text = True)
                printMessages(mdrun_output)

                energies = open(f'{outPath}/{ligand}_md.log', 'rt')
                energies = energies.readlines()
                ene_dict = {'Bond': zero, 'Angle': zero, 'Proper Dih.': zero, 
                            'LJ-14': zero, 'Coulomb-14': zero, 
                            'LJ (SR)': zero, 'Coulomb (SR)': zero, 
                            'Potential': zero} 
                for i, line in enumerate(energies):
                    if line.strip().startswith('Energies'):
                        keys = [s.strip() for s in energies[i+1].split('  ') if s]
                        values = energies[i+2].split()
                        while keys[0]:
                            for key, val in zip(keys, values):
                                ene_dict[key] = float(val) * unit.kilojoules_per_mole
                            i += 2
                            keys = [s.strip() for s in energies[i+1].split('  ') if s]
                            values = energies[i+2].split()
                count += 1            
            print(ene_dict)
            canonical = canonicalize_energy_names(ene_dict, gromacs_to_canonical)
            pprint(canonical, indent = 4)
            gromacsEnergies[target][ligand] = canonical
                        
            # energy minimization
            minEnergies = ''
            count = 0
            while not minEnergies and count < 10:
                print('    Energy minimization')
                print('    Messages of gmx grompp:')
                grompp_output = subprocess.run(f'gmx grompp -f ./input/gromacs/minim.mdp \
                                               -c {ligPath}/{ligand}.gro \
                                               -p {ligPath}/{ligand}.top \
                                               -o {outPath}/{ligand}_em.tpr \
                                               -maxwarn 1'.split(), 
                                               capture_output = True, text = True)
                printMessages(grompp_output)
                
                print('    Messages of gmx mdrun:')
                mdrun_output = subprocess.run(f'gmx mdrun  -s {outPath}/{ligand}_em.tpr \
                                               -deffnm {outPath}/{ligand}_em'.split(),
                                               capture_output = True, text = True)
                printMessages(mdrun_output)
                
                # getting the last energies entry
                minEnergies = open(f'{outPath}/{ligand}_em.log', 'rt')
                minEnergies = minEnergies.readlines()
                minEne_dict = {'Bond': zero, 'Angle': zero, 'Proper Dih.': zero, 
                               'LJ-14': zero, 'Coulomb-14': zero, 
                               'LJ (SR)': zero, 'Coulomb (SR)': zero, 
                               'Potential': zero}
                for i, line in enumerate(minEnergies):
                    if line.strip().startswith('Energies'):
                        keys = [s.strip() for s in minEnergies[i+1].split('  ') if s]
                        values = minEnergies[i+2].split()
                        while keys[0]:
                            for key, val in zip(keys, values):
                                minEne_dict[key] = float(val) * unit.kilojoules_per_mole
                            i += 2
                            keys = [s.strip() for s in minEnergies[i+1].split('  ') if s]
                            values = minEnergies[i+2].split()
                count += 1
            print(minEne_dict)
            canonical = canonicalize_energy_names(minEne_dict, gromacs_to_canonical)
            pprint(canonical, indent = 4)
            gromacsMinimizedEnergies[target][ligand] = canonical

#### Amber

Single point energies calculated with AMBER. The handling of output files is adapted from https://github.com/shirtsgroup/InterMol

In [ ]:
amberEnergies = {}
amberMinimizedEnergies = {}
for target in ['testmols',  'jnk1', 'pde2', 'thrombin', 'pde2_sdf']:
    amberEnergies[target] = {}
    amberMinimizedEnergies[target] = {}
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    ligands = ! ls -d 'systems/'$target/*/
    for lig in ligands:
        lig = lig.split('/')
        ligand = lig[-2]
        ligPath = 'systems/' + target + '/' + ligand + '/'
        outPath = 'systems/' + target + '/' + ligand + '/amber/'
        
        if os.path.isfile(ligPath+ligand+'.inpcrd') and os.path.isfile(ligPath+ligand+'.prmtop'):
            print('-' * 60)
            print('--- ' + ligand + ' ---')
            print('-' * 60)
            ! mkdir -p $outPath
            ! rm -f $outPath/*.*
            ! rm -f $ligPath/\#* $outPath/\#*
            
            # single point energy of starting structure
            energies = ''
            count = 0
            while not energies and count < 10:
                print('    Calculate single-point energy ...')
                print('    Message of pemd:')
                pemd_output = subprocess.run(f'{os.getenv("AMBERHOME")}/bin/pmemd.cuda \
                                               -O \
                                               -i input/amber/min.in \
                                               -o {outPath}/{ligand}.o \
                                               -p {ligPath}/{ligand}.prmtop \
                                               -c {ligPath}/{ligand}.inpcrd \
                                               -r {outPath}/{ligand}.rst \
                                               -inf {outPath}/{ligand}.mdinfo \
                                               '.split(), 
                                               capture_output = True, text = True)
                print('    | ' + pemd_output.stdout + pemd_output.stderr)
                                
                with open(f'{outPath}/{ligand}.o', 'rt') as f:
                    energies = f.readlines()
        
                # Find where the energy information starts.
                for i, line in enumerate(energies):
                    if line[0:8] == '   NSTEP':
                        startline = i
                        break
                else:
                    raise AmberError('Unable to detect where energy info starts in AMBER '
                                         'output file: {}'.format(mdout))

                # Strange ranges for amber file data.
                ranges = [[1, 24], [26, 49], [51, 77]]

                e_out = {}
                potential = 0 * unit.kilocalories_per_mole
                for line in energies[startline+3:]:
                    if '=' in line:
                        for i in range(3):
                            r = ranges[i]
                            term = line[r[0]:r[1]]
                            if '=' in term:
                                energy_type, energy_value = term.split('=')
                                energy_value = float(energy_value) * unit.kilocalories_per_mole
                                potential += energy_value
                                energy_type = energy_type.rstrip()
                                e_out[energy_type] = energy_value
                    else:
                        break
                e_out['ENERGY'] = potential
                
                canonical = canonicalize_energy_names(e_out, amber_to_canonical)
                amberEnergies[target][ligand] = canonical 
                pprint(canonical, indent = 4)

# Summarize results

#### Single point energies
The different columns are the openMM, GROMACS, AMBER energies, and GROMACS-openMM and AMBER-openMM energy differences. All values in kJ/mol. 

In [ ]:
energies = {}
differing_molecules = []
print(f'{"Energy Contrib.":20s}:{"OpenMM":>15s}{"GROMACS":>15s}{"AMBER":>15s}{"GRO-OMM":>15s}{"AMB-OMM":>15s}')
for target in ['testmols', 'jnk1', 'pde2', 'pde2_sdf', 'thrombin']:
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    energies[target] = {}
    for ligand in openmmEnergies[target].keys():
        print('-' * 60)
        print('--- ' + ligand + ' ---')
        print('-' * 60)    
        energies[target][ligand] = {}
        for kk, ind in zip(canonical_energy_names, canonical_indent):
            omm_ene = openmmEnergies[target][ligand][kk].value_in_unit(unit.kilojoules_per_mole)
            gro_ene = gromacsEnergies[target][ligand][kk].value_in_unit(unit.kilojoules_per_mole)
            amb_ene = amberEnergies[target][ligand][kk].value_in_unit(unit.kilojoules_per_mole)
            diff_gro_omm = gro_ene - omm_ene
            diff_amb_omm = amb_ene - omm_ene
            if kk == 'potential' and (np.fabs(diff_gro_omm) > 1e-2 or np.fabs(diff_amb_omm) > 1e-2):
                differing_molecules.append([target, ligand])
            energies[target][ligand][kk]=np.array([omm_ene, gro_ene, amb_ene])
            if ind < 2:
                print(f'\033[1m{" " * 2 *  ind + kk:20s}:{omm_ene:15.5e}{gro_ene:15.5e}{amb_ene:15.5e}{diff_gro_omm:15.5e}{diff_amb_omm:15.5e} \033[0m')
            else:
                print(f'{" " * 2 *  ind + kk:20s}:{omm_ene:15.5e}{gro_ene:15.5e}{amb_ene:15.5e}{diff_gro_omm:15.5e}{diff_amb_omm:15.5e}')

#### Molecules with deviation > 0.01 kJ/mol in the total potential energy

In [ ]:
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules

print(f'{"Energy Contrib.":20s}:{"OpenMM":>15s}{"GROMACS":>15s}{"AMBER":>15s}{"GRO-OMM":>15s}{"AMB-OMM":>15s}')
rdkitMols = []
for i, (target, ligand) in enumerate(np.unique(differing_molecules, axis=0)):
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    ligPath = 'systems/' + target + '/' + ligand + '/' + ligand
        
    if os.path.isfile(ligPath+'.sdf'):
        print('-' * 60)
        print('--- ' + ligand + ' ---')
        print('-' * 60)
        mols = AllChem.SDMolSupplier(ligPath+'.sdf')
        for mol in mols:
            AllChem.Compute2DCoords(mol)
            mol.SetProp('_Name', target + ': ' + ligand)
            rdkitMols.append(mol)
    for (kk, item), ind in zip(energies[target][ligand].items(), canonical_indent):
        if ind < 2:
            print(f'\033[1m{" " * 2 *  ind + kk:20s}:{item[0]:15.5e}{item[1]:15.5e}{item[2]:15.5e}{item[1]-item[0]:15.5e}{item[2]-item[0]:15.5e} \033[0m')
        else:
            print(f'{" " * 2 *  ind + kk:20s}:{item[0]:15.5e}{item[1]:15.5e}{item[2]:15.5e}{item[1]-item[0]:15.5e}{item[2]-item[0]:15.5e}')
Draw.MolsToGridImage(rdkitMols, legends=[x.GetProp('_Name') for x in rdkitMols])

#### Minimized Energies

In [ ]:
minEnergies={}
print(f'{"Energy Contrib.":20s}:{"OpenMM":>15s}{"GROMACS":>15s}{"AMBER":>15s}{"GRO-OMM":>15s}{"AMB-OMM":>15s}')
for target in ['testmols', 'jnk1', 'pde2', 'pde2_sdf', 'thrombin']:
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    minEnergies[target] = {}
    for ligand in openmmMinimizedEnergies[target].keys():
        print('-' * 60)
        print('--- ' + ligand + ' ---')
        print('-' * 60)
        minEnergies[target][ligand] = {}
        for kk, ind in zip(canonical_energy_names, canonical_indent):
            omm_ene = openmmEnergies[target][ligand][kk].value_in_unit(unit.kilojoules_per_mole)
            gro_ene = gromacsEnergies[target][ligand][kk].value_in_unit(unit.kilojoules_per_mole)
#             amb_ene = amberEnergies[target][ligand][kk].value_in_unit(unit.kilojoules_per_mole)
            diff_gro_omm = gro_ene - omm_ene
#             diff_amb_omm = amb_ene - omm_ene
#             energies[target][ligand][kk]=np.array([omm_ene, gro_ene, amb_ene])
#             print('    {:20s}: {:15.5e}    {:15.5e}    {:15.5e}    {:15.5e}    {:15.5e}'.format(kk, omm_ene, gro_ene, amb_ene, diff_gro_omm, diff_gro_omm))
            minEnergies[target][ligand][kk]=np.array([omm_ene, gro_ene])
            if ind < 2:
                print(f'\033[1m{" " * 2 *  ind + kk:20s}:{omm_ene:15.5e}{gro_ene:15.5e}{amb_ene:15.5e}{diff_gro_omm:15.5e}{diff_amb_omm:15.5e} \033[0m')
            else:
                print(f'{" " * 2 *  ind + kk:20s}:{omm_ene:15.5e}{gro_ene:15.5e}{amb_ene:15.5e}{diff_gro_omm:15.5e}{diff_amb_omm:15.5e}') 

# Calculate RMSD between the minimized structures 
Between Gromacs and OpenMM. So far without roto-translational alignment

In [ ]:
for target in ['testmols',  'jnk1', 'pde2', 'thrombin', 'pde2_sdf']:
    print('=' * 60)
    print('=== ' + target + ' ===')
    print('=' * 60)
    ligands = ! ls -d 'systems/'$target/*/
    for lig in ligands:
        lig = lig.split('/')
        ligPath = 'systems/' + target + '/' + lig[-2] + '/' + lig[-2]
        outPath = 'systems/' + target + '/' + lig[-2] + '/gromacs/' + lig[-2]
        if os.path.isfile(ligPath+'_minimized.pdb') and os.path.isfile(outPath+'_em.gro'):
            print('-' * 60)
            print('--- ' + lig[-2] + ' ---')
            print('-' * 60)
            gromacs_positions = pmd.load_file(outPath+'_em.gro')
            openmm_positions = pmd.load_file(ligPath+'_minimized.pdb')
#             print(type(gromacs_positions.positions.value_in_unit(unit.nanometer)), type(openmm_positions.positions.value_in_unit(unit.nanometer)))
#             print(gromacs_positions.positions.value_in_unit(unit.nanometer), openmm_positions.positions.value_in_unit(unit.nanometer))
#             print(np.array(gromacs_positions.positions.value_in_unit(unit.nanometer))-np.array(openmm_positions.positions.value_in_unit(unit.nanometer)))
#             print(np.square(np.array(gromacs_positions.positions.value_in_unit(unit.nanometer))-np.array(openmm_positions.positions.value_in_unit(unit.nanometer))))
            squared_distances = np.sum(np.square(np.array(gromacs_positions.positions.value_in_unit(unit.nanometer))-np.array(openmm_positions.positions.value_in_unit(unit.nanometer))), axis = 1)
#             print(squared_distances)
            print('    RMSD: {:.4f} nm'.format(np.sqrt(squared_distances.sum()/squared_distances.shape[0])))  